In [1]:
import pandas as pd
import numpy as np
import pathlib as pl
import sklearn
import pathlib as pl

In [2]:
path = '../dataset/finRep/FinancialPhraseBank/all-data.csv'
df = pd.read_csv(path, encoding = "ISO-8859-1", names = ['sentiment', 'sentence'])
print(df[:10])

  sentiment                                           sentence
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...
5  positive  FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
6  positive  For the last quarter of 2010 , Componenta 's n...
7  positive  In the third quarter of 2010 , net sales incre...
8  positive  Operating profit rose to EUR 13.1 mn from EUR ...
9  positive  Operating profit totalled EUR 21.1 mn , up fro...


In [3]:
# to install keybert
# !pip install nltk
# !pip install keybert
# !pip install sentence-transformers
# !pip install git+https://github.com/LIAAD/yake
### To show the execution times
# !pip install ipython-autotime
# %load_ext autotime

In [18]:
# choose a random sample
text = df.sample(1)['sentence'].iloc[0]
label = df.sample(1)['sentiment'].iloc[0]
print(text)
print("the sentiment is : %s"%label)

`` Because we 're a pension insurance company , we 're required to diversify and not put too much in one asset class .
the sentiment is : negative


Rake

In [5]:
from rake_nltk import Rake
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

In [7]:
# Import them
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
sent_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
keyBERT_model = KeyBERT(model = sent_trans)

In [9]:
def extract_terms(document, n_gram_range = (3,3), 
                  top_N=5, model=keyBERT_model, diversity_threshold = 0.7):

  keywords = model.extract_keywords(document, stop_words='english', 
                                    keyphrase_ngram_range=(1, 3),
                                    use_mmr=True, 
                                    diversity = diversity_threshold,
                                    top_n = top_N)
  
  return sorted(keywords, key=lambda tup:(-tup[1], tup[0]))

In [10]:
print(f'Text: {text}.\nLength: {len(text.split())}')
best_terms = extract_terms(text)

for terms_and_score in best_terms:
  print(terms_and_score)

Text: The Marubeni Group focuses on creating `` value chain '' from upstream to downstream , encompassing a wide variety of business fields , including oil & gas , metals , mineral resources , foods , pulp & paper and chemicals , among others ..
Length: 44
('value chain upstream', 0.7109)
('marubeni', 0.4173)
('resources foods pulp', 0.2288)
('encompassing wide variety', 0.2081)
('gas metals', 0.1838)


In [11]:
import yake
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(text)

for kw in keywords:
	print(kw)


print("SECOND FORM:")
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

NLTK-VADER

In [27]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
test_subset=['20170412', 'great', 'bad', 'terrible', 'dog', 'stop', 'good','prices','increasing']

sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for word in test_subset:
    if (sid.polarity_scores(word)['compound']) >= 0.5:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.5:
        neg_word_list.append(word)
    else:
        neu_word_list.append(word)                

print('Positive :',pos_word_list)        
print('Neutral :',neu_word_list)    
print('Negative :',neg_word_list)

Positive : ['great']
Neutral : ['20170412', 'terrible', 'dog', 'stop', 'good', 'prices', 'increasing']
Negative : ['bad']


In [25]:
from textblob import TextBlob

def word_polarity(test_subset):
    pos_word_list=[]
    neu_word_list=[]
    neg_word_list=[]

    for word in test_subset:               
        testimonial = TextBlob(word)
        if testimonial.sentiment.polarity >= 0.5:
            pos_word_list.append(word)
        elif testimonial.sentiment.polarity <= -0.5:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)

    print('Positive :',pos_word_list)        
    print('Neutral :',neu_word_list)    
    print('Negative :',neg_word_list)      

word_polarity(['20170412', 'great', 'bad', 'terrible', 'prices', 'increasing', 'good','bad','fat'])

Positive : ['great', 'good']
Neutral : ['20170412', 'prices', 'increasing', 'fat']
Negative : ['bad', 'terrible', 'bad']
